# Estimating the biomass of livestock
To estimate the biomass of livestock, we rely on data on global stocks of cattle, sheep goats, and pigs fro the Food and Agriculture Organization database FAOStat. We downloaded data from the domain Production/Live animals.
We combined data on the total stocks of each animal with estimates of the mean mass of each type of animal species from [Dong et al.](http://www.ipcc-nggip.iges.or.jp/public/2006gl/pdf/4_Volume4/V4_10_Ch10_Livestock.pdf), Annex 10A.2, Tables 10A-4 to 10A-9.

Here are samples of the data:

In [120]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:,.1e}'.format
# Load global stocks data
stocks = pd.read_csv('FAOSTAT_stock_data_mammals.csv')
stocks.head()

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,QA,Live Animals,5100,Africa,5111,Stocks,1107,Asses,2014,2014,Head,18946358,A,"Aggregate, may include official, semi-official..."
1,QA,Live Animals,5100,Africa,5111,Stocks,946,Buffaloes,2014,2014,Head,3949287,A,"Aggregate, may include official, semi-official..."
2,QA,Live Animals,5100,Africa,5111,Stocks,1126,Camels,2014,2014,Head,23533724,A,"Aggregate, may include official, semi-official..."
3,QA,Live Animals,5100,Africa,5111,Stocks,866,Cattle,2014,2014,Head,312327289,A,"Aggregate, may include official, semi-official..."
4,QA,Live Animals,5100,Africa,5111,Stocks,1016,Goats,2014,2014,Head,374380445,A,"Aggregate, may include official, semi-official..."


In [104]:
# Load species body mass data
body_mass = pd.read_excel('livestock_body_mass.xlsx',skiprows=1,index_col=0) 
body_mass.head()

,Cattle - dairy,Cattle - non-dairy,Buffaloes,Swine - market,Swine - breeding,Sheep,Goats,Horses,Asses,Mules,Camels,"Camelids, other"
IPCC Area,,,,,,,,,,,,
Indian Subcontinent,275,110,295,28,28,28.0,30.0,238,130,130,217,217
Eastern Europe,550,391,380,50,180,48.5,38.5,377,130,130,217,217
Africa,275,173,380,28,28,28.0,30.0,238,130,130,217,217
Oceania,500,330,380,45,180,48.5,38.5,377,130,130,217,217
Western Europe,600,420,380,50,198,48.5,38.5,377,130,130,217,217


We pivot the stocks DataFrame to have a view of each kind of animal at each region:

In [98]:
# Replace NaN with zeros
stocks.fillna(value=0,inplace=True)
stock_pivot = pd.pivot(stocks.Area,stocks.Item, stocks.Value).astype(float)

# Replace NaN with zeros
stock_pivot.fillna(value=0,inplace=True)

stock_pivot

Item,Asses,Buffaloes,"Camelids, other",Camels,Cattle,Goats,Horses,Mules,Pigs,Sheep
Area,,,,,,,,,,
Africa,18946358.0,3949287.0,0.0,23533724.0,312327289.0,374380445.0,6058131.0,1023087.0,34332061.0,340749117.0
Americas,6801642.0,1326495.0,8894140.0,0.0,508942003.0,35640927.0,32501406.0,5937682.0,169902095.0,86074410.0
Asia,16490197.0,188792665.0,0.0,4194792.0,491019874.0,580703222.0,14316963.0,2960413.0,590547756.0,536250670.0
Eastern Europe,103491.0,16608.0,0.0,7425.0,39614609.0,4593553.0,2759863.0,3508.0,52939645.0,36459697.0
Northern America,52006.0,0.0,0.0,0.0,100746715.0,2641350.0,10669150.0,4001.0,80831900.0,6139950.0
Oceania,9000.0,245.0,0.0,0.0,40226178.0,3992930.0,404766.0,0.0,5345671.0,102431992.0
Southern Asia,8301000.0,151629876.0,0.0,1683000.0,269230027.0,293219445.0,1323601.0,583500.0,11309552.0,153316034.0
Western Europe,34083.0,6507.0,0.0,0.0,42210298.0,2051113.0,1099252.0,31445.0,64895542.0,10800347.0


There is a difference between the body mass of a dairy producing cow to a non-dairy producing cow. We thus count seperately the dairy producing cattle from the non-dairy producing cattle. Data about the amount of dairy cattle comes from the FAOStat domain Production - Livestock Primary.
There is also a difference in body mass between breeding and non-breeding pigs. We assume 90% of the population is breeding based on IPCC, 2006, Vol.4, Ch.10,Table.10.19.

In [99]:
# Load data on the number of dairy producing cattle
dairy = pd.read_csv('FAOSTAT_cattle_dairy_data.csv')

# Set the index of the DataFrame to be the region so we can compare with the stocks data
dairy.set_index('Area',inplace=True)

# Add a category of dairy producing cattle
stock_pivot['Cattle - dairy'] = dairy.Value

# Set the amount of non-dairy producing cattle to be the total number minus the dairy producing cattle
stock_pivot['Cattle'] = stock_pivot['Cattle']-stock_pivot['Cattle - dairy']

# Rename the Cattle column name to Cattle - non-dairy
stock_pivot.rename(columns={'Cattle': 'Cattle - non-dairy'}, inplace=True)

# Set the amount of non-breeding (market) pigs (swine) to 10% of the total amount of pigs
stock_pivot['Swine - market'] = 0.1*stock_pivot['Pigs']

# Set the amount of breeding pigs (swine) to 90% of the total amount of pigs
stock_pivot['Pigs'] *= 0.9

# Rename the Pigs column name to Swine - breeding
stock_pivot.rename(columns={'Pigs': 'Swine - breeding'}, inplace=True)

stock_pivot

Item,Asses,Buffaloes,"Camelids, other",Camels,Cattle - non-dairy,Goats,Horses,Mules,Swine - breeding,Sheep,Cattle - dairy,Swine - market
Area,,,,,,,,,,,,
Africa,18946358.0,3949287.0,0.0,23533724.0,244890721.0,374380445.0,6058131.0,1023087.0,30898854.9,340749117.0,67436568,3433206.1
Americas,6801642.0,1326495.0,8894140.0,0.0,454011484.0,35640927.0,32501406.0,5937682.0,152911885.5,86074410.0,54930519,16990209.5
Asia,16490197.0,188792665.0,0.0,4194792.0,383448681.0,580703222.0,14316963.0,2960413.0,531492980.4,536250670.0,107571193,59054775.6
Eastern Europe,103491.0,16608.0,0.0,7425.0,23425833.0,4593553.0,2759863.0,3508.0,47645680.5,36459697.0,16188776,5293964.5
Northern America,52006.0,0.0,0.0,0.0,90585405.0,2641350.0,10669150.0,4001.0,72748710.0,6139950.0,10161310,8083190.0
Oceania,9000.0,245.0,0.0,0.0,33351427.0,3992930.0,404766.0,0.0,4811103.9,102431992.0,6874751,534567.1
Southern Asia,8301000.0,151629876.0,0.0,1683000.0,199904964.0,293219445.0,1323601.0,583500.0,10178596.8,153316034.0,69325063,1130955.2
Western Europe,34083.0,6507.0,0.0,0.0,30920889.0,2051113.0,1099252.0,31445.0,58405987.8,10800347.0,11289409,6489554.2


Data on the mass of animals is divided into different regions than the FAOStat data so we need preprocess the stocks DataFrame and merge it with the body mass data:

In [100]:
# Preprocessing the stocks DataFrame

# Calculate the total number of animals in Latin America by subtracting values for Northern America from the total
# values for the Americas
stock_pivot.loc['Americas'] -= stock_pivot.loc['Northern America']

# Change name of Americas to Latin America
stock_pivot.rename(index={'Americas': 'Latin America'},inplace=True)

# Calculate the total number of animals in Asia without the Indian Subcontinent by subtracting values for the Southern Asia 
# from the total values for the Asia
stock_pivot.loc['Asia'] -= stock_pivot.loc['Southern Asia']

# Change name of Southern Asia to Indian Subcontinent
stock_pivot.rename(index={'Southern Asia': 'Indian Subcontinent'},inplace=True)


stock_pivot


Item,Asses,Buffaloes,"Camelids, other",Camels,Cattle - non-dairy,Goats,Horses,Mules,Swine - breeding,Sheep,Cattle - dairy,Swine - market
Area,,,,,,,,,,,,
Africa,18946358.0,3949287.0,0.0,23533724.0,244890721.0,374380445.0,6058131.0,1023087.0,30898854.9,340749117.0,67436568.0,3433206.1
Latin America,6749636.0,1326495.0,8894140.0,0.0,363426079.0,32999577.0,21832256.0,5933681.0,80163175.5,79934460.0,44769209.0,8907019.5
Asia,8189197.0,37162789.0,0.0,2511792.0,183543717.0,287483777.0,12993362.0,2376913.0,521314383.6,382934636.0,38246130.0,57923820.4
Eastern Europe,103491.0,16608.0,0.0,7425.0,23425833.0,4593553.0,2759863.0,3508.0,47645680.5,36459697.0,16188776.0,5293964.5
Northern America,52006.0,0.0,0.0,0.0,90585405.0,2641350.0,10669150.0,4001.0,72748710.0,6139950.0,10161310.0,8083190.0
Oceania,9000.0,245.0,0.0,0.0,33351427.0,3992930.0,404766.0,0.0,4811103.9,102431992.0,6874751.0,534567.1
Indian Subcontinent,8301000.0,151629876.0,0.0,1683000.0,199904964.0,293219445.0,1323601.0,583500.0,10178596.8,153316034.0,69325063.0,1130955.2
Western Europe,34083.0,6507.0,0.0,0.0,30920889.0,2051113.0,1099252.0,31445.0,58405987.8,10800347.0,11289409.0,6489554.2


We now multiply the stocks of each animal type and for each region by the characteristic body weight of each animal:

In [121]:
wet_biomass =(body_mass*stock_pivot)
wet_biomass

,Asses,Buffaloes,"Camelids, other",Camels,Cattle - dairy,Cattle - non-dairy,Goats,Horses,Mules,Sheep,Swine - breeding,Swine - market
Africa,2.5e+09,1.5e+09,0.0e+00,5.1e+09,1.9e+10,4.2e+10,1.1e+10,1.4e+09,1.3e+08,9.5e+09,8.7e+08,9.6e+07
Asia,1.1e+09,1.4e+10,0.0e+00,5.5e+08,1.3e+10,7.2e+10,1.1e+10,4.9e+09,3.1e+08,1.9e+10,9.4e+10,2.9e+09
Eastern Europe,1.3e+07,6.3e+06,0.0e+00,1.6e+06,8.9e+09,9.2e+09,1.8e+08,1.0e+09,4.6e+05,1.8e+09,8.6e+09,2.6e+08
Indian Subcontinent,1.1e+09,4.5e+10,0.0e+00,3.7e+08,1.9e+10,2.2e+10,8.8e+09,3.2e+08,7.6e+07,4.3e+09,2.9e+08,3.2e+07
Latin America,8.8e+08,5.0e+08,1.9e+09,0.0e+00,1.8e+10,1.1e+11,9.9e+08,5.2e+09,7.7e+08,2.2e+09,2.2e+09,2.5e+08
Middle east,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Northern America,6.8e+06,0.0e+00,0.0e+00,0.0e+00,6.1e+09,3.5e+10,1.0e+08,4.0e+09,5.2e+05,3.0e+08,1.4e+10,3.7e+08
Oceania,1.2e+06,9.3e+04,0.0e+00,0.0e+00,3.4e+09,1.1e+10,1.5e+08,1.5e+08,0.0e+00,5.0e+09,8.7e+08,2.4e+07
Western Europe,4.4e+06,2.5e+06,0.0e+00,0.0e+00,6.8e+09,1.3e+10,7.9e+07,4.1e+08,4.1e+06,5.2e+08,1.2e+10,3.2e+08


We sum over all regions and convert units from kg wet weight to Gt C carbon by assuming carbon is ≈15% of the wet weight (30% dry weight of wet weight and carbon is 50% of dry weight).

In [126]:
# conversion factor from kg wet weight to Gt C
kg_to_gt_c = 1000*0.15/1e15
total_biomass = wet_biomass.sum()*kg_to_gt_c
best_estimate = total_biomass.sum()
print('Our best estimate for the biomass of mammal livestock is %.1f Gt C' % best_estimate)

Our best estimate for the biomass of mammal livestock is 0.1 Gt C
